This is the implementation of Approach 1 for Transient Requirement validation (Section 4.2.1). Users need to read and run [Transient_base.ipynb](./Transient_base.ipynb) before reading and runing this notebook.

Here we compare either the parameterized time series (linear + seasonal + offsets + exponential components) from the InSAR stack or the individual unwrapped interferograms with cGNSS, across the length scales described in the L2 requirements. We first calculate the difference of either the relevant time series parameters (e.g., secular deformation rate for Requirement 658, or co-seismic displacement for Requirement 660) or the interferogram displacement (for Requirement 663) between all possible pairs of cGNSS locations within a validation region. This gives the vectors Δmest,cGNSS and Δmest,InSAR, along with the baseline distance L between locations. We then calculate the residual differences between vectors Δmest,cGNSS and Δmest,InSAR , bin them by baseline distance L, and obtain the percentage of the absolute value of residuals in each bin that fall below the thresholds defined in each of the SES L2 requirements. (Note: the threshold is a single value for the secular rate requirement, and a baseline-dependent curve for the coseismic and transient requirements). 
We assume that the residuals between cGNSS and InSAR follow a Gaussian distribution. For each distance bin, if the fraction of residuals lying below the bin threshold value is more than 0.683 (i.e., one standard deviation), we judge the derived secular deformation rate or co-seismic deformation to pass the corresponding requirement. Requirement 663 for transient deformation requires that 70% of the individual interferograms pass the threshold curve. 


In [6]:
import numpy as np
import pickle
from pathlib import Path
import pandas as pd

In [7]:
calval_dir = Path.cwd()/'calval'
calval_location = 'central_valley'
work_dir = calval_dir/calval_location

In [8]:
with open(work_dir/'approach1.pkl','rb') as f:
    dist, rel_measure, ifgs_date = pickle.load(f)

In [9]:
n_ifgs = len(dist)

In [10]:
if n_ifgs == 1:
    print("Validation approach 1 for Secular or Coseismic.")
else:
    print("validation approach 1 for Transient.")

validation approach 1 for Transient.


In [11]:
n_bins = 10
bins = np.linspace(0.1,50.0,num=n_bins+1)

In [12]:
n_all = np.empty([n_ifgs,n_bins+1],dtype=int) # number of points for each ifgs and bins
n_pass = np.empty([n_ifgs,n_bins+1],dtype=int) # number of points pass
#ratio = np.empty([n_ifgs,n_bins+1]) # ratio
# the final column is the ratio as a whole
for i in range(n_ifgs):
    inds = np.digitize(dist[i],bins)
    for j in range(1,n_bins+1):
        rqmt = 3*(1+np.sqrt(dist[i][inds==j]))# mission requirement for i-th ifgs and j-th bins
        rem = rel_measure[i][inds==j] # relative measurement
        assert len(rqmt) == len(rem)
        n_all[i,j-1] = len(rem)
        n_pass[i,j-1] = np.count_nonzero(rem<rqmt)
    n_all[i,-1] = np.sum(n_all[i,0:-2])
    n_pass[i,-1] = np.sum(n_pass[i,0:-2])

In [13]:
def to_str(x:bool):
    if x==True:
        return 'true '
    elif x==False:
        return 'false '

In [14]:
ratio = n_pass/n_all
thresthod = 0.683 
#The assumed nature of Gaussian distribution gives a probability of 0.683 of being within one standard deviation.
success_or_fail = ratio>thresthod
success_or_fail_str = [list(map(to_str, x)) for x in success_or_fail]

In [15]:
columns = []
for i in range(n_bins):
    columns.append(f'{bins[i]:.2f}-{bins[i+1]:.2f}')
columns.append('total')

In [16]:
index = []
for i in range(len(ifgs_date)):
    index.append(ifgs_date[i,0].strftime('%Y%m%d')+'-'+ifgs_date[i,1].strftime('%Y%m%d'))

In [17]:
n_all_pd = pd.DataFrame(n_all,columns=columns,index=index)
n_pass_pd = pd.DataFrame(n_pass,columns=columns,index=index)
ratio_pd = pd.DataFrame(ratio,columns=columns,index=index)
success_or_fail_pd = pd.DataFrame(success_or_fail_str,columns=columns,index=index)

Number of data points in each bin:

In [18]:
n_all_pd

,0.10-5.09,5.09-10.08,10.08-15.07,15.07-20.06,20.06-25.05,25.05-30.04,30.04-35.03,35.03-40.02,40.02-45.01,45.01-50.00,total
20190110-20190122,36,87,100,122,119,114,142,150,116,109,986
20190203-20190215,39,93,105,122,119,112,136,132,108,99,966
20190227-20190311,37,95,104,120,113,109,137,136,107,97,958
20190323-20190404,42,89,106,123,121,123,151,157,134,119,1046
20190416-20190428,41,99,106,132,128,128,155,154,125,120,1068
20190510-20190522,46,97,108,132,126,125,156,163,137,122,1090
20190603-20190615,34,92,103,125,120,120,149,151,129,115,1023
20190627-20190709,44,102,111,135,131,132,163,169,137,124,1124
20190721-20190802,41,96,106,121,118,115,140,142,116,110,995
20190814-20190826,39,88,104,118,119,118,147,154,127,119,1014


Number of data points that below the curve:

In [19]:
n_pass_pd

,0.10-5.09,5.09-10.08,10.08-15.07,15.07-20.06,20.06-25.05,25.05-30.04,30.04-35.03,35.03-40.02,40.02-45.01,45.01-50.00,total
20190110-20190122,22,68,91,115,111,113,140,147,115,107,922
20190203-20190215,29,71,75,96,95,97,118,109,92,74,782
20190227-20190311,22,64,73,80,76,73,82,69,53,44,592
20190323-20190404,39,79,89,104,94,101,122,130,100,85,858
20190416-20190428,37,84,79,82,78,99,115,115,84,65,773
20190510-20190522,40,75,89,104,102,103,122,125,101,77,861
20190603-20190615,29,82,91,111,108,116,136,132,113,99,918
20190627-20190709,37,77,61,67,60,57,63,69,62,58,553
20190721-20190802,37,93,89,79,76,82,95,100,79,79,730
20190814-20190826,36,75,92,104,111,109,136,136,111,105,910


Percentage of pass:

In [20]:
s = ratio_pd.style
s.set_table_styles([  # create internal CSS classes
    {'selector': '.true', 'props': 'background-color: #e6ffe6;'},
    {'selector': '.false', 'props': 'background-color: #ffe6e6;'},
], overwrite=False)
s.set_td_classes(success_or_fail_pd)

,0.10-5.09,5.09-10.08,10.08-15.07,15.07-20.06,20.06-25.05,25.05-30.04,30.04-35.03,35.03-40.02,40.02-45.01,45.01-50.00,total
20190110-20190122,0.611111,0.781609,0.910000,0.942623,0.932773,0.991228,0.985915,0.980000,0.991379,0.981651,0.935091
20190203-20190215,0.743590,0.763441,0.714286,0.786885,0.798319,0.866071,0.867647,0.825758,0.851852,0.747475,0.809524
20190227-20190311,0.594595,0.673684,0.701923,0.666667,0.672566,0.669725,0.598540,0.507353,0.495327,0.453608,0.617954
20190323-20190404,0.928571,0.887640,0.839623,0.845528,0.776860,0.821138,0.807947,0.828025,0.746269,0.714286,0.820268
20190416-20190428,0.902439,0.848485,0.745283,0.621212,0.609375,0.773438,0.741935,0.746753,0.672000,0.541667,0.723783
20190510-20190522,0.869565,0.773196,0.824074,0.787879,0.809524,0.824000,0.782051,0.766871,0.737226,0.631148,0.789908
20190603-20190615,0.852941,0.891304,0.883495,0.888000,0.900000,0.966667,0.912752,0.874172,0.875969,0.860870,0.897361
20190627-20190709,0.840909,0.754902,0.549550,0.496296,0.458015,0.431818,0.386503,0.408284,0.452555,0.467742,0.491993
20190721-20190802,0.902439,0.968750,0.839623,0.652893,0.644068,0.713043,0.678571,0.704225,0.681034,0.718182,0.733668
20190814-20190826,0.923077,0.852273,0.884615,0.881356,0.932773,0.923729,0.925170,0.883117,0.874016,0.882353,0.897436


In [21]:
ratio_pd.iloc[0]['total']

0.9350912778904665

In [22]:
success_or_fail_pd

,0.10-5.09,5.09-10.08,10.08-15.07,15.07-20.06,20.06-25.05,25.05-30.04,30.04-35.03,35.03-40.02,40.02-45.01,45.01-50.00,total
20190110-20190122,false,true,true,true,true,true,true,true,true,true,true
20190203-20190215,true,true,true,true,true,true,true,true,true,true,true
20190227-20190311,false,false,true,false,false,false,false,false,false,false,false
20190323-20190404,true,true,true,true,true,true,true,true,true,true,true
20190416-20190428,true,true,true,false,false,true,true,true,false,false,true
20190510-20190522,true,true,true,true,true,true,true,true,true,false,true
20190603-20190615,true,true,true,true,true,true,true,true,true,true,true
20190627-20190709,true,true,false,false,false,false,false,false,false,false,false
20190721-20190802,true,true,true,false,false,true,false,true,false,true,true
20190814-20190826,true,true,true,true,true,true,true,true,true,true,true


In [23]:
if n_ifgs == 1:
    print("Validation approach 1 for Secular or Coseismic:")
    if success_or_fail_pd.iloc[0]['total']:
        print("This velocity dataset passes the requirement.")
    else:
        print("This velocity dataset does not pass the requirement.")
else:
    percentage = np.count_nonzero(ratio_pd['total']>thresthod)/n_ifgs
    print(f"Percentage of interferograms passes the requirement: {percentage}")

Percentage of interferograms passes the requirement: 0.8333333333333334
